In [1]:
from sklearn.linear_model import SGDRegressor
model = SGDRegressor()
model.partial_fit([[0,0]],[0])
model.predict([[0,0]])

array([0.])

In [2]:
model.partial_fit([[1,0]],[1])

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [3]:
model.predict([[5,0]])

array([0.05045378])

In [ ]:
import gym
import numpy as np
gamma = 0.9
env = gym.make('MountainCar-v0')
observation = env.reset()

def get_transformer():
    sampler = RBFSampler(gamma=1, n_components=500, random_state=1234)
    observation_sample = np.vstack([env.observation_space.sample() for i in range(1000)])
    return sampler.fit(observation_sample)

transformer = get_transformer()
transformed_observation = transformer.transform([observation])
print('transformed_observation.shape',transformed_observation.shape)
models = [SGDRegressor() for i in range(3)]
for model in models:
    model.partial_fit(transformed_observation,[0])

def run(env, render = False):
    observation = env.reset()
    done = False
    total_reward = 0
    while(done == False):        
        action = None
                    
        transformed_observation = transformer.transform([observation])
        
        predictions = [m.predict(transformed_observation) for m in models]
        
        highest_index = np.hstack([i for i,p in enumerate(predictions) if p == max(predictions)])
#         print('highest_index',highest_index)
        action = int(np.random.choice(highest_index))
#         print('action',action)
        next_observation, reward, done, info = env.step(action)
        
        next_transformed_observation = transformer.transform([next_observation])
        
        next_predictions = [m.predict(next_transformed_observation) for m in models]
        new_value = reward + gamma * max(next_predictions)
        models[action].partial_fit(next_transformed_observation, new_value)
        
        
        if(render):
            env.render()
        total_reward += reward
        
    if(render):
        env.close()
    return total_reward 

for i in range(100):
    print(np.mean([run(env) for i in range(100)]))
#     run(env, render=True)

transformed_observation.shape (1, 500)
-200.0
-200.0


In [22]:
from sklearn.kernel_approximation import RBFSampler


RBFSampler(gamma=1, n_components=500, random_state=1234)

In [39]:
run(env, render=True)

highest_index [array([0.]), array([0.]), array([0.])]


ValueError: a must be 1-dimensional